# Michele Banfi 869294

The exercise consists of a quantum circuit in which is present a "black box"; the black box apply some CNOTs, the target of the black box is an ancilla qubit. The aim of the exercise is guess which are the qubit acting as the controller qubit inside CNOTs.

In [1]:
# import libraries
from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler as Sampler
import numpy as np

A black box function is used. Randomly, it applies some CNOTs with the ancilla as the target qubit.

In [22]:
# black box
def blackBox():
    # save the indices:
    indices = []
    for i in range(4):
        random = np.random.randint(2)
        if random == 1:
            indices.append(i)
    
    return indices

A classical approach and a quantum approach are used to solve the exercise.

# Classical

For the classical approach, first compute the permutations (with only one qubit = 1 at a time), cycle trhough all of them; then observe the outputs of the ancilla throughout the permutations to guess which qubits are controlling CNOTs.

In [23]:
permutations = []

# calculate the permutations of the inputs except the last qubit
for i in range(4):
    configuration = []
    for j in range(4):
        if i != j:
            configuration.append(0)
        else:
            configuration.append(1)
    permutations.append(configuration)

print(permutations)

[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]


In [24]:
# save the results to inspect them afterward
results = []
indices = blackBox()

# cycle through the permutations
for i in range(len(permutations)):
    
    classical_input = permutations[i]
    
    qc = QuantumCircuit(5)
    
    # encode the state
    for i, val in enumerate(classical_input):
        if val == 1:
            qc.x(i)
    
    # black box
    for i in indices:
        qc.cx(i, 4)
    
    # measure the circuit
    qc.measure_all()
    
    # run the circuit
    result = Sampler().run([qc], shots=100).result()[0]
    co = result.data.meas.get_counts()

    # save the results to inspect them later
    results.append(co)


In [26]:
# print the circuit to showcase how it works
print(qc)

                        ░ ┌─┐            
   q_0: ───────■────────░─┤M├────────────
               │        ░ └╥┘┌─┐         
   q_1: ───────┼────────░──╫─┤M├─────────
               │        ░  ║ └╥┘┌─┐      
   q_2: ───────┼────────░──╫──╫─┤M├──────
        ┌───┐  │        ░  ║  ║ └╥┘┌─┐   
   q_3: ┤ X ├──┼────■───░──╫──╫──╫─┤M├───
        └───┘┌─┴─┐┌─┴─┐ ░  ║  ║  ║ └╥┘┌─┐
   q_4: ─────┤ X ├┤ X ├─░──╫──╫──╫──╫─┤M├
             └───┘└───┘ ░  ║  ║  ║  ║ └╥┘
meas: 5/═══════════════════╩══╩══╩══╩══╩═
                           0  1  2  3  4 


In [25]:
# cycle through the results 
for i in range(len(results)):
    result = list(results[i].keys())[0]
    
    # check that the string at position 0 is 1, which is the ancilla; meaning that there was a CNOT using the qubit as a control over the ancilla
    if result[0] == '1':
        print(f"The circuit had a CNOT at position {i}")

The circuit had a CNOT at position 0
The circuit had a CNOT at position 3


# Quantum

Now the quantum version

In [27]:
# initialize the circuit
qc = QuantumCircuit(5)

# put the ancilla in state |1> that will become |-> after the Hadamard
qc.x(4)

# prepare the state |+>
for i in range(5):
    qc.h(i)

# black box
indices = blackBox()
for i in indices:
    qc.cx(i, 4)

# put back in computational basis
for i in range(5):
    qc.h(i)
    
# put back the ancilla in state |0>
qc.x(4)

qc.measure_all()

print(qc)

        ┌───┐          ┌───┐           ░ ┌─┐            
   q_0: ┤ H ├───────■──┤ H ├───────────░─┤M├────────────
        ├───┤┌───┐  │  └───┘           ░ └╥┘┌─┐         
   q_1: ┤ H ├┤ H ├──┼──────────────────░──╫─┤M├─────────
        ├───┤├───┤  │                  ░  ║ └╥┘┌─┐      
   q_2: ┤ H ├┤ H ├──┼──────────────────░──╫──╫─┤M├──────
        ├───┤└───┘  │       ┌───┐      ░  ║  ║ └╥┘┌─┐   
   q_3: ┤ H ├───────┼────■──┤ H ├──────░──╫──╫──╫─┤M├───
        ├───┤┌───┐┌─┴─┐┌─┴─┐├───┤┌───┐ ░  ║  ║  ║ └╥┘┌─┐
   q_4: ┤ X ├┤ H ├┤ X ├┤ X ├┤ H ├┤ X ├─░──╫──╫──╫──╫─┤M├
        └───┘└───┘└───┘└───┘└───┘└───┘ ░  ║  ║  ║  ║ └╥┘
meas: 5/══════════════════════════════════╩══╩══╩══╩══╩═
                                          0  1  2  3  4 


In [28]:
result = Sampler().run([qc], shots=10).result()[0] #note the [0]
co = result.data.meas.get_counts()

# take the result key 
result = list(co.keys())[0]

for i in range (len(result)):
    if result[4-i] == '1':
        print(f"The circuit had a CNOT at position {i}")

The circuit had a CNOT at position 0
The circuit had a CNOT at position 3
